In [2]:
# Load data from EDA
import pandas as pd
train_df = pd.read_parquet("../data/train_clean.parquet")
valid_df = pd.read_parquet("../data/valid_clean.parquet")
holdout_df = pd.read_parquet("../data/holdout_clean.parquet")

In [3]:
print(train_df.shape); print(valid_df.shape); print(holdout_df.shape)

(7278841, 86)
(1819711, 86)
(1605627, 86)


### Evaluation Protocol & Metrics

**Goal.** Evaluate models for both **ranking performance** (under class imbalance) and **probability calibration**.

**Data Split.**
- All model tuning, selection, and calibration are performed on **`train_df` only**, using **3-fold stratified cross-validation** (`N_FOLDS=3`).
- Both `valid_df` and `holdout_df` are strictly held out and used **only for final evaluation/reporting**.

**Primary Metric.**
- **AP (Average Precision / PR-AUC):** Captures ranking quality under strong class imbalance (positive rate ≈ 1.9%).

**Calibration Metrics.**
- **LogLoss:** Evaluates probability calibration quality.
- **WLL (50:50 Weighted LogLoss):** Adjusts for class imbalance to allow fairer model comparison.

**Leakage Policy.**
- No information from `valid_df` or `holdout_df` is used during training, tuning, or early stopping.  
  All model decisions are made **only from CV results on `train_df`**.



In [5]:
# =========================================================
# Setup 
# =========================================================
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, log_loss
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from pandas.api.types import CategoricalDtype

SEED = 42
np.random.seed(SEED)

# All CV later must use train-only folds
N_FOLDS = 3

TARGET_COL = "clicked"
SEQ_COL    = "seq"  # excluded from modeling
CAT_COLS_FIXED = [
    "gender","age_group","inventory_id","day_of_week","hour",
    "l_feat_1","l_feat_2","l_feat_3","l_feat_4","l_feat_8","l_feat_13",
    "l_feat_16","l_feat_18","l_feat_19","l_feat_20","l_feat_21",
    "l_feat_22","l_feat_23","l_feat_24","l_feat_26","l_feat_27"
]

# --- helpers
def cast_categoricals(df: pd.DataFrame, cat_cols):
    out = df.copy()
    for c in cat_cols:
        if c in out.columns and not isinstance(out[c].dtype, CategoricalDtype):
            out[c] = out[c].astype("category")
    return out

train_df = cast_categoricals(train_df, CAT_COLS_FIXED)
valid_df = cast_categoricals(valid_df, CAT_COLS_FIXED)
if "holdout_df" in locals():
    holdout_df = cast_categoricals(holdout_df, CAT_COLS_FIXED)

# =========================================================
# Fixed feature set (exclude target & seq). Tuning uses ONLY train via K-fold CV.
# =========================================================
EXCLUDE   = {TARGET_COL, SEQ_COL}
feat_cols = [c for c in train_df.columns if c not in EXCLUDE]

X_tr, y_tr = train_df[feat_cols], train_df[TARGET_COL].astype(int).to_numpy()
X_va, y_va = valid_df[feat_cols], valid_df[TARGET_COL].astype(int).to_numpy()
X_ho       = holdout_df[feat_cols] if "holdout_df" in locals() else None
y_ho       = (holdout_df[TARGET_COL].astype(int).to_numpy()
              if ("holdout_df" in locals() and TARGET_COL in holdout_df) else None)

# =========================================================
# Metrics helpers (AP, LogLoss, WLL)
# =========================================================
def weighted_log_loss(y_true, p, pos_weight=0.5, neg_weight=0.5, eps=1e-15):
    p = np.clip(np.asarray(p, dtype=float), eps, 1.0 - eps)
    y = np.asarray(y_true, dtype=int)
    n_pos = max(int((y == 1).sum()), 0)
    n_neg = max(int((y == 0).sum()), 0)
    w_pos = (pos_weight / n_pos) if n_pos > 0 else 0.0
    w_neg = (neg_weight / n_neg) if n_neg > 0 else 0.0
    sw = np.where(y == 1, w_pos, w_neg)
    return float(log_loss(y, p, sample_weight=sw, labels=[0, 1]))

def competition_score(ap, wll):
    return float(ap - wll)

def compute_metrics(y_true, p):
    ap  = average_precision_score(y_true, p)
    nll = log_loss(y_true, p)
    wll = weighted_log_loss(y_true, p, 0.5, 0.5)
    return {"AP": ap, "LogLoss": nll, "WLL": wll}

# audit class ratios
def _ratio(y):
    y = np.asarray(y)
    return f"pos={y.mean():.6f}, neg={1.0 - y.mean():.6f}"

print("[Class ratios]")
print("  train  :", _ratio(y_tr))
print("  valid  :", _ratio(y_va))
print("  holdout:", _ratio(y_ho) if y_ho is not None else "N/A (labels missing)")


[Class ratios]
  train  : pos=0.019075, neg=0.980925
  valid  : pos=0.019074, neg=0.980926
  holdout: pos=0.019075, neg=0.980925


In [9]:
# =========================================================
# Baseline comparison (LogReg vs. LightGBM vs. XGBoost) on TRAIN via K-fold CV only
# - No leakage: uses only train_df (no valid/holdout)
# - Metrics: LogLoss, AP (PR-AUC), WLL(50:50)
# =========================================================
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, log_loss
from sklearn.utils import check_random_state
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import xgboost as xgb

BASE_N = 300_000
RNG = check_random_state(SEED)

# helpers
def stratified_sample(df: pd.DataFrame, n: int, ycol: str, seed: int) -> pd.DataFrame:
    if (n is None) or (n >= len(df)):
        return df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    pos = df[df[ycol] == 1]
    neg = df[df[ycol] == 0]
    rate = pos.shape[0] / max(len(df), 1)
    n_pos = max(1, int(round(n * rate)))
    n_neg = max(1, n - n_pos)
    pos_s = pos.sample(min(n_pos, len(pos)), random_state=seed)
    neg_s = neg.sample(min(n_neg, len(neg)), random_state=seed)
    return pd.concat([pos_s, neg_s]).sample(frac=1.0, random_state=seed).reset_index(drop=True)

def cast_categoricals_inplace(df: pd.DataFrame, cat_cols: list[str]):
    for c in cat_cols:
        if c in df.columns and not pd.api.types.is_categorical_dtype(df[c]):
            df[c] = df[c].astype("category")

def cat_to_codes(df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    out = df.copy()
    for c in [c for c in cat_cols if c in out.columns]:
        if pd.api.types.is_categorical_dtype(out[c]):
            out[c] = out[c].cat.codes
        elif out[c].dtype.name == "object":
            out[c] = out[c].astype("category").cat.codes
    return out

# features
EXCLUDE = {TARGET_COL, SEQ_COL}
feat_cols = [c for c in train_df.columns if c not in EXCLUDE]

# training subset for fast CV baseline
tr_s  = stratified_sample(train_df, BASE_N, TARGET_COL, SEED)
Xtr_s = tr_s[feat_cols].copy()
ytr_s = tr_s[TARGET_COL].astype(int).to_numpy()

# cleanup
obj_cols_tr = Xtr_s.select_dtypes(include=['object']).columns.tolist()
if obj_cols_tr:
    Xtr_s.drop(columns=obj_cols_tr, inplace=True, errors="ignore")
cat_present = [c for c in CAT_COLS_FIXED if c in feat_cols]
cast_categoricals_inplace(Xtr_s, cat_present)

# Logistic Regression (OHE baseline)
num_cols = [c for c in feat_cols if c not in cat_present]
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

prep_lr = ColumnTransformer(
    transformers=[
        ("cat", ohe, [c for c in cat_present if c in Xtr_s.columns]),
        ("num", SimpleImputer(strategy="mean"), [c for c in num_cols if c in Xtr_s.columns]),
    ],
    sparse_threshold=0.3
)

lr = Pipeline([
    ("prep", prep_lr),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
])

# LightGBM default baseline (per-fold scale_pos_weight)
lgbm_base = dict(
    n_estimators=2000,
    learning_rate=0.05,
    objective="binary",
    random_state=SEED,
    n_jobs=-1
)

# XGBoost default baseline (per-fold scale_pos_weight)
xgb_base = dict(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    random_state=SEED,
    n_jobs=-1,
    early_stopping_rounds=100  # XGBoost early stopping을 모델 초기화 시 설정
)

# K-fold CV on train only
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

def _cv_eval(model_name: str):
    ap_scores, wll_scores, nll_scores = [], [], []
    fold_idx = 0
    for tr_idx, va_idx in skf.split(Xtr_s, ytr_s):
        fold_idx += 1
        Xtr, Xcv = Xtr_s.iloc[tr_idx], Xtr_s.iloc[va_idx]
        ytr, ycv = ytr_s[tr_idx], ytr_s[va_idx]

        print(f"[{model_name}] Fold {fold_idx}: train pos={ytr.mean():.6f}, cv pos={ycv.mean():.6f}")

        if model_name == "LogReg(OHE)":
            model = lr
            Xtr_fit, Xcv_fit = Xtr, Xcv

        elif model_name == "LightGBM(Default)":
            params = dict(lgbm_base)
            pos_tr = int(ytr.sum()); neg_tr = len(ytr) - pos_tr
            params["scale_pos_weight"] = float(neg_tr / max(pos_tr, 1))
            model = lgb.LGBMClassifier(**params)
            Xtr_fit, Xcv_fit = Xtr, Xcv

        elif model_name == "XGBoost(Default)":
            params = dict(xgb_base)
            pos_tr = int(ytr.sum()); neg_tr = len(ytr) - pos_tr
            params["scale_pos_weight"] = float(neg_tr / max(pos_tr, 1))
            model = xgb.XGBClassifier(**params)
            Xtr_fit = cat_to_codes(Xtr, cat_present)
            Xcv_fit = cat_to_codes(Xcv, cat_present)

        else:
            raise ValueError(f"Unknown model_name={model_name}")

        # ---- fit per model ----
        if model_name == "LogReg(OHE)":
            model.fit(Xtr_fit, ytr)

        elif model_name == "LightGBM(Default)":
            model.fit(
                Xtr_fit, ytr,
                eval_set=[(Xcv_fit, ycv)],
                eval_metric="binary_logloss",
                callbacks=[lgb.early_stopping(100, verbose=False)]
            )

        elif model_name == "XGBoost(Default)":
            model.fit(
                Xtr_fit, ytr,
                eval_set=[(Xcv_fit, ycv)],
                verbose=False
            )

        p = model.predict_proba(Xcv_fit)[:, 1] if hasattr(model, "predict_proba") else model.predict(Xcv_fit)
        ap_scores.append(average_precision_score(ycv, p))
        wll_scores.append(weighted_log_loss(ycv, p, 0.5, 0.5))
        nll_scores.append(log_loss(ycv, p))

    return {
        "AP_mean": float(np.mean(ap_scores)),   "AP_std": float(np.std(ap_scores)),
        "WLL_mean": float(np.mean(wll_scores)), "WLL_std": float(np.std(wll_scores)),
        "LogLoss_mean": float(np.mean(nll_scores)), "LogLoss_std": float(np.std(nll_scores)),
    }

res_lr  = _cv_eval("LogReg(OHE)")
res_lgb = _cv_eval("LightGBM(Default)")
res_xgb = _cv_eval("XGBoost(Default)")

# report (primary sort by AP, then WLL)
baseline_df = (
    pd.DataFrame([
        {"Model": "LogReg(OHE)",        **res_lr},
        {"Model": "LightGBM(Default)",  **res_lgb},
        {"Model": "XGBoost(Default)",   **res_xgb},
    ])
    .sort_values(["AP_mean", "WLL_mean"], ascending=[False, True])
    .reset_index(drop=True)
)

_fmt = {
    "AP_mean":"{:.6f}", "AP_std":"{:.6f}",
    "WLL_mean":"{:.6f}", "WLL_std":"{:.6f}",
    "LogLoss_mean":"{:.6f}", "LogLoss_std":"{:.6f}",
}
cols = ["Model","AP_mean","AP_std","WLL_mean","WLL_std","LogLoss_mean","LogLoss_std"]

print("Baseline (TRAIN K-fold CV only) — lower WLL/LogLoss are better; higher AP is better")
display(baseline_df[cols].style.format(_fmt))

/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_6192/1119675554.py:37: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in df.columns and not pd.api.types.is_categorical_dtype(df[c]):


[LogReg(OHE)] Fold 1: train pos=0.019070, cv pos=0.019080


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LogReg(OHE)] Fold 2: train pos=0.019075, cv pos=0.019070


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_6192/1119675554.py:43: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(out[c]):


[XGBoost(Default)] Fold 3: train pos=0.019075, cv pos=0.019070


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_6192/1119675554.py:43: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(out[c]):


Baseline (TRAIN K-fold CV only) — lower WLL/LogLoss are better; higher AP is better


,Model,AP_mean,AP_std,WLL_mean,WLL_std,LogLoss_mean,LogLoss_std
0,LightGBM(Default),0.042527,0.001616,1.333233,0.004809,0.110837,0.000338
1,XGBoost(Default),0.040019,0.001335,2.125304,0.028454,0.104368,0.000467
2,LogReg(OHE),0.039996,0.001288,0.664000,0.001439,0.655804,0.002371


### Baseline Model Comparison (Train CV Only)

Three models were compared using 3-fold CV on the training data:

Interpretation:
- LightGBM achieved the best balance between ranking performance (AP) and calibrated loss (WLL).
- XGBoost yielded slightly better log-loss but suffered from overconfident predictions (high WLL).
- LogReg performed competitively in WLL but lagged in AP and LogLoss.

Decision:

- Despite XGBoost showing decent performance, LightGBM was selected for further tuning due to its faster training and memory efficiency on large-scale data (~10M rows), aligning better with computational constraints.


### LightGBM Tuning Plan (Leak-Free, Train-CV Only)

**Objective**  
Maximize **PR-AUC (AP)** under class imbalance.  
Ensure good probability calibration via **LogLoss** and **WLL (50:50)**.  
Final evaluation will report all three metrics for robustness.

---

**Leak-Free Tuning Protocol**

- All tuning (Optuna search, early stopping, model selection) uses only `train_df` via **K-fold CV**.
- **Isotonic calibration** is also fit using **OOF predictions from train folds only**.
- `valid_df` and `holdout_df` are **never used** during tuning or calibration.
- After tuning:
  - Best config is trained on **full train_df**
  - Isotonic calibration is applied
  - Evaluation is run **once on holdout_df**

---

**Tuning Stages**

| Stage | Purpose                | Trials  | Description |
|------:|-------------------------|--------:|-------------|
| 1     | Broad exploration       | 100–200 | Tune tree size, regularization, sampling, `scale_pos_weight` vs `is_unbalance` |
| 2     | Focused exploitation    | 30–50   | Zoom into top 10% trials from Stage 1 |
| 3     | Final model & calibration | 1     | Train best config on full train, calibrate with OOF isotonic, evaluate once on holdout |

---

**Reporting**

- **Baseline Comparison (Train CV):** LogReg(OHE) vs LightGBM(Default)
- **Tuning Summary:** Top trials sorted by CV-AP and CV-LogLoss
- **Final Holdout Evaluation:** AP, LogLoss, WLL, and predicted probability statistics

---

**Reproducibility**

- Controlled by top-level constants: `SEED`, `N_FOLDS`, optional subsample sizes.
- All artifacts (trials, params, models, calibration objects, holdout predictions) saved to:
  `./artifacts/optuna_lgbm/`


In [21]:
# =========================================================
# Phase 1 — Hyperparameter tuning (Optuna, broad search)
# - Goal: maximize AP (PR-AUC) while monitoring LogLoss
# - Data: larger stratified subset for a stable search
# =========================================================
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from sklearn.metrics import average_precision_score, log_loss
from sklearn.model_selection import StratifiedKFold

TUNE_N = 1_000_000  

def stratified_sample(df: pd.DataFrame, n: int, ycol: str, seed: int) -> pd.DataFrame:
    """Stratified sample preserving class prior."""
    if (n is None) or (n >= len(df)):
        return df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    pos = df[df[ycol] == 1]
    neg = df[df[ycol] == 0]
    rate = pos.shape[0] / max(len(df), 1)
    n_pos = max(1, int(round(n * rate)))
    n_neg = max(1, n - n_pos)
    pos_s = pos.sample(min(n_pos, len(pos)), random_state=seed)
    neg_s = neg.sample(min(n_neg, len(neg)), random_state=seed)
    return pd.concat([pos_s, neg_s]).sample(frac=1.0, random_state=seed).reset_index(drop=True)

# --- build tuning subset (train only); valid set stays fixed (X_va, y_va from above)
tr_tune = stratified_sample(train_df, TUNE_N, TARGET_COL, SEED)
Xt, yt = tr_tune[feat_cols].copy(), tr_tune[TARGET_COL].astype(int).to_numpy()

# --- defensive: drop stray object cols and cast categories for LightGBM
obj_cols_t = Xt.select_dtypes(include=['object']).columns.tolist()
if obj_cols_t:
    Xt.drop(columns=obj_cols_t, inplace=True, errors="ignore")
cat_present = [c for c in CAT_COLS_FIXED if c in Xt.columns]
for c in cat_present:
    if not pd.api.types.is_categorical_dtype(Xt[c]):
        Xt[c] = Xt[c].astype('category')

def objective_lgbm(trial: optuna.trial.Trial) -> float:
    """3-fold Stratified CV with early stopping. Optimize for AP (higher is better)."""
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 1023, log=True),
        "max_depth": trial.suggest_int("max_depth", 6, 15),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 5000, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": 1,
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "verbosity": -1,
        "random_state": SEED,
        "n_jobs": -1,
    }

    # imbalance handling toggle (compare is_unbalance vs scale_pos_weight)
    use_spw = trial.suggest_categorical("use_scale_pos_weight", [True, False])
    if use_spw:
        pos = yt.sum(); neg = len(yt) - pos
        params["scale_pos_weight"] = float(neg / max(pos, 1))
    else:
        params["is_unbalance"] = True

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    ap_scores, nll_scores, wll_scores = [], [], []

    for tr_idx, va_idx in skf.split(Xt, yt):
        Xtr, Xcv = Xt.iloc[tr_idx], Xt.iloc[va_idx]
        ytr, ycv = yt[tr_idx], yt[va_idx]

        clf = lgb.LGBMClassifier(
            n_estimators=4000,
            **params
        )
        clf.fit(
            Xtr, ytr,
            eval_set=[(Xcv, ycv)],
            eval_metric=['binary_logloss', 'aucpr'],
            callbacks=[lgb.early_stopping(150, verbose=False), lgb.log_evaluation(0)]
        )

        it = getattr(clf, "best_iteration_", None)
        p = clf.predict_proba(Xcv, num_iteration=it)[:, 1] if it else clf.predict_proba(Xcv)[:, 1]

        ap_scores.append(average_precision_score(ycv, p))
        nll_scores.append(log_loss(ycv, p))
        # record WLL(50:50)
        wll_scores.append(weighted_log_loss(ycv, p, 0.5, 0.5))

    trial.set_user_attr("cv_logloss", float(np.mean(nll_scores)))
    # store mean CV WLL
    trial.set_user_attr("cv_wll", float(np.mean(wll_scores)))
    # Optuna minimizes → negative AP
    return -float(np.mean(ap_scores))

study = optuna.create_study(direction="minimize", study_name="lgbm_tune_ap")
study.optimize(objective_lgbm, n_trials=120, show_progress_bar=True)

# --- compact Top‑K trial table for the notebook
df_trials = study.trials_dataframe()
topk = (df_trials.assign(AP=-df_trials["value"],
                        CV_LogLoss=df_trials["user_attrs_cv_logloss"],
                        CV_WLL=df_trials["user_attrs_cv_wll"])
                .sort_values("value")
                .head(10))

display(topk[[
    "number","AP","CV_LogLoss","CV_WLL",
    "params_learning_rate","params_num_leaves","params_max_depth",
    "params_min_data_in_leaf","params_feature_fraction","params_bagging_fraction",
    "params_lambda_l1","params_lambda_l2","params_use_scale_pos_weight"
]])


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/3042281080.py:38: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xt[c]):
[I 2025-09-14 14:36:57,610] A new study created in memory with name: lgbm_tune_ap
Best trial: 0. Best value: -0.0492498:   1%|          | 1/120 [00:24<49:05, 24.75s/it]

[I 2025-09-14 14:37:22,369] Trial 0 finished with value: -0.04924975312172609 and parameters: {'learning_rate': 0.05872180976996895, 'num_leaves': 69, 'max_depth': 8, 'min_data_in_leaf': 569, 'feature_fraction': 0.637113266198345, 'bagging_fraction': 0.9918973453550037, 'lambda_l1': 3.9862455750819157, 'lambda_l2': 0.7214066901273297, 'use_scale_pos_weight': False}. Best is trial 0 with value: -0.04924975312172609.


Best trial: 0. Best value: -0.0492498:   2%|▏         | 2/120 [06:36<7:30:32, 229.09s/it]

[I 2025-09-14 14:43:34,495] Trial 1 finished with value: -0.04660062608922394 and parameters: {'learning_rate': 0.1637086811099485, 'num_leaves': 48, 'max_depth': 7, 'min_data_in_leaf': 104, 'feature_fraction': 0.829322617938997, 'bagging_fraction': 0.8675948017354721, 'lambda_l1': 9.630881697033352, 'lambda_l2': 5.385645428983777, 'use_scale_pos_weight': False}. Best is trial 0 with value: -0.04924975312172609.


Best trial: 0. Best value: -0.0492498:   2%|▎         | 3/120 [06:55<4:19:43, 133.19s/it]

...

Best trial: 116. Best value: -0.0514933:  99%|█████████▉| 119/120 [2:25:14<02:10, 130.53s/it]

[I 2025-09-14 17:02:12,570] Trial 118 finished with value: -0.04471465452595144 and parameters: {'learning_rate': 0.14116612976969922, 'num_leaves': 129, 'max_depth': 6, 'min_data_in_leaf': 295, 'feature_fraction': 0.8803759209024561, 'bagging_fraction': 0.9756731550402419, 'lambda_l1': 0.687372726397689, 'lambda_l2': 0.2265775822330701, 'use_scale_pos_weight': True}. Best is trial 116 with value: -0.051493306140401846.


Best trial: 116. Best value: -0.0514933: 100%|██████████| 120/120 [2:30:52<00:00, 75.43s/it] 

[I 2025-09-14 17:07:49,801] Trial 119 finished with value: -0.04438980645096601 and parameters: {'learning_rate': 0.1615598070637498, 'num_leaves': 109, 'max_depth': 6, 'min_data_in_leaf': 329, 'feature_fraction': 0.8522831979347928, 'bagging_fraction': 0.9902331626558243, 'lambda_l1': 0.18933989015597008, 'lambda_l2': 4.146234459824428, 'use_scale_pos_weight': True}. Best is trial 116 with value: -0.051493306140401846.


,number,AP,CV_LogLoss,CV_WLL,params_learning_rate,params_num_leaves,params_max_depth,params_min_data_in_leaf,params_feature_fraction,params_bagging_fraction,params_lambda_l1,params_lambda_l2,params_use_scale_pos_weight
116,116,0.051493,0.449692,0.682932,0.132467,120,6,292,0.851944,0.985067,0.511101,0.200551,True
103,103,0.051476,0.225058,0.848284,0.096762,149,6,337,0.817208,0.891574,0.604707,0.231233,True
114,114,0.051463,0.295005,0.761026,0.110475,175,6,397,0.830057,0.992668,0.682654,0.148936,True
61,61,0.051400,0.285600,0.780256,0.108829,261,7,417,0.850602,0.808620,0.926678,0.016827,True
113,113,0.051360,0.286404,0.769472,0.109011,179,6,371,0.831894,0.986830,0.598349,0.385152,True
97,97,0.051355,0.327663,0.746671,0.116004,121,7,239,0.817566,0.959346,1.355391,0.235934,True
85,85,0.051240,0.400360,0.697256,0.126220,142,6,438,0.769533,0.962202,2.966409,0.020805,False
111,111,0.051188,0.389832,0.702236,0.124858,158,6,370,0.834908,0.956685,0.864355,0.167959,True
101,101,0.051117,0.325229,0.737773,0.115529,149,6,450,0.818621,0.913775,1.281032,0.232988,True
62,62,0.051115,0.264355,0.802758,0.104995,142,7,454,0.840994,0.814785,1.077938,0.311351,True


In [23]:
# ==========================================================================
# Validation Gate (safe: no in-place mutation, baseline first, OOF isotonic)
# ==========================================================================
import json, copy, numpy as np, pandas as pd
from pathlib import Path
from joblib import dump
import lightgbm as lgb
from sklearn.metrics import average_precision_score, log_loss
from sklearn.model_selection import StratifiedKFold

# --- helpers ---
def compute_metrics(y_true, p):
    return {"AP": average_precision_score(y_true, p),
            "LogLoss": log_loss(y_true, p),
            "WLL": weighted_log_loss(y_true, p, 0.5, 0.5)
            }

def prepare_lgb_frames(X_tr_src, X_va_src, cat_cols):
    X_tr_clean = X_tr_src.copy()
    X_va_clean = X_va_src.copy()
    obj_tr = X_tr_clean.select_dtypes(include=['object']).columns.tolist()
    obj_va = X_va_clean.select_dtypes(include=['object']).columns.tolist()
    if obj_tr: X_tr_clean.drop(columns=obj_tr, inplace=True, errors="ignore")
    if obj_va: X_va_clean.drop(columns=obj_va, inplace=True, errors="ignore")
    for c in [c for c in cat_cols if c in X_tr_clean.columns]:
        if not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
            X_tr_clean[c] = X_tr_clean[c].astype("category")
    for c in [c for c in cat_cols if c in X_va_clean.columns]:
        if not pd.api.types.is_categorical_dtype(X_va_clean[c]):
            X_va_clean[c] = X_va_clean[c].astype("category")
    return X_tr_clean, X_va_clean

ART_DIR = Path("./artifacts/optuna_lgbm"); ART_DIR.mkdir(parents=True, exist_ok=True)
def _save_df(df, name): df.to_csv(ART_DIR / f"{name}.csv", index=False)
def _save_json(obj, name): (ART_DIR / f"{name}.json").write_text(json.dumps(obj, indent=2))

# --- clean frames (no in-place) ---
X_tr_clean, X_va_clean = prepare_lgb_frames(X_tr, X_va, CAT_COLS_FIXED)

# --- baseline first (independent) ---
lgb_base = lgb.LGBMClassifier(
    n_estimators=2000, learning_rate=0.05, objective="binary",
    is_unbalance=True, random_state=SEED, n_jobs=-1
)
lgb_base.fit(
    X_tr_clean, y_tr,
    eval_set=[(X_va_clean, y_va)],
    eval_metric="binary_logloss",
    callbacks=[lgb.early_stopping(100, verbose=False)]
)
p_base = lgb_base.predict_proba(X_va_clean)[:, 1]
m_base = compute_metrics(y_va, p_base)

# --- best params (no mutation of originals) ---
best = copy.deepcopy(study.best_trial.params)
use_spw = best.pop("use_scale_pos_weight", True)
if use_spw:
    pos = int(y_tr.sum()); neg = len(y_tr) - pos
    best["scale_pos_weight"] = float(neg / max(pos, 1))
else:
    best["is_unbalance"] = True
best.setdefault("bagging_freq", 1)
best.update(dict(objective="binary", metric="binary_logloss", random_state=SEED, n_jobs=-1))
_save_json(best, "best_params_valid_gate")

# --- OOF isotonic calibration ---
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
oof_pred = np.zeros_like(y_tr, dtype=float)
for tr_idx, va_idx in skf.split(X_tr_clean, y_tr):
    clf_cv = lgb.LGBMClassifier(n_estimators=4000, **best)
    clf_cv.fit(
        X_tr_clean.iloc[tr_idx], y_tr[tr_idx],
        eval_set=[(X_tr_clean.iloc[va_idx], y_tr[va_idx])],
        eval_metric=["binary_logloss","aucpr"],
        callbacks=[lgb.early_stopping(150, verbose=False)]
    )
    oof_pred[va_idx] = clf_cv.predict_proba(X_tr_clean.iloc[va_idx])[:, 1]

from sklearn.isotonic import IsotonicRegression
iso = IsotonicRegression(y_min=0.0, y_max=1.0, out_of_bounds="clip")
iso.fit(oof_pred, y_tr)
dump(iso, ART_DIR / "isotonic_stage1.joblib")

# --- full-train fit (uncalibrated) + calibrated eval on the SAME valid set ---
lgb_best = lgb.LGBMClassifier(n_estimators=6000, **best)
lgb_best.fit(
    X_tr_clean, y_tr,
    eval_set=[(X_va_clean, y_va)],
    eval_metric=["binary_logloss","aucpr"],
    callbacks=[lgb.early_stopping(200, verbose=False)]
)
p_uncal = lgb_best.predict_proba(X_va_clean)[:, 1]
p_cal   = iso.transform(p_uncal)

m_uncal = compute_metrics(y_va, p_uncal)
m_cal   = compute_metrics(y_va, p_cal)

# --- report (no NaNs) ---
cmp = pd.DataFrame([
    {"Model": "LightGBM(Default Baseline)",    **m_base,  "MeanPred": float(p_base.mean())},
    {"Model": "Best(Stage-1) Uncalibrated",    **m_uncal, "MeanPred": float(p_uncal.mean())},
    {"Model": "Best(Stage-1) + Isotonic(OOF)", **m_cal,   "MeanPred": float(p_cal.mean())},
]).sort_values(["LogLoss","AP"], ascending=[True, False]).reset_index(drop=True)
display(cmp)

# --- gating on calibrated metrics ---
delta_ap = m_cal["AP"] - m_base["AP"]
delta_ll = m_cal["LogLoss"] - m_base["LogLoss"]  # negative is better
print(f"Base rate (valid) = {y_va.mean():.6f}")
print(f"[Gate] ΔAP = {delta_ap:+.6f}, ΔLogLoss = {delta_ll:+.6f} (negative is better)")

# --- artifacts ---
_save_df(cmp, "valid_gate_comparison_calibrated_safe")
np.save(ART_DIR / "p_valid_baseline.npy", p_base)
np.save(ART_DIR / "p_valid_uncal.npy", p_uncal)
np.save(ART_DIR / "p_valid_calibrated.npy", p_cal)


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/552147930.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/552147930.py:29: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(X_va_clean[c]):


,Model,AP,LogLoss,WLL,MeanPred
0,Best(Stage-1) + Isotonic(OOF),0.055251,0.088486,1.835594,0.019183
1,LightGBM(Default Baseline),0.049019,0.110668,1.317379,0.060929
2,Best(Stage-1) Uncalibrated,0.055589,0.447979,0.669499,0.310960


Base rate (valid) = 0.019074
[Gate] ΔAP = +0.006233, ΔLogLoss = -0.022182 (negative is better)


### Validation Gate (report-only, no selection)

- After Stage‑1 tuning on **train CV only**, we fit **isotonic calibration** using **train OOF** and evaluated **once** on `valid`.
- Results vs. default LightGBM:
  - **AP**: +0.0062 (better ranking under imbalance)
  - **LogLoss**: −0.0222 (reduced over‑confidence; better probability quality)
  - **WLL (50:50)**: increased (expected when calibration lowers probabilities toward the base rate; positive-class term −log(p) grows)
- Mean predicted probability moved from **0.311 (uncal.) → 0.019 (cal.)**, matching the base rate (~0.019). This confirms **successful probability calibration**.

We will proceed to Stage‑2


In [24]:
# =========================================================
# Stage-2 — Adaptive narrow tuning from Stage-1 signals
# =========================================================
import json, copy
import numpy as np, pandas as pd, optuna, lightgbm as lgb
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, log_loss

ART_DIR = Path("./artifacts/optuna_lgbm/stage2"); ART_DIR.mkdir(parents=True, exist_ok=True)

# --- reusable helpers (same as before) ---------------------------------------
def stratified_sample(df, n, ycol, seed):
    if (n is None) or (n >= len(df)): 
        return df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    pos = df[df[ycol] == 1]; neg = df[df[ycol] == 0]
    r = pos.shape[0] / max(len(df),1); n_pos = max(1, int(round(n*r))); n_neg = n - n_pos
    return pd.concat([
        pos.sample(min(n_pos, len(pos)), random_state=seed),
        neg.sample(min(n_neg, len(neg)), random_state=seed)
    ]).sample(frac=1.0, random_state=seed).reset_index(drop=True)

def make_clean_frames(train_df, valid_df, feat_cols, cat_cols):
    X_tr_clean = train_df[feat_cols].copy()
    X_va_clean = valid_df[feat_cols].copy()
    y_tr_clean = train_df[TARGET_COL].astype(int).to_numpy()
    y_va_clean = valid_df[TARGET_COL].astype(int).to_numpy()
    for df in (X_tr_clean, X_va_clean):
        obj = df.select_dtypes(include=["object"]).columns.tolist()
        if obj: df.drop(columns=obj, inplace=True, errors="ignore")
    for c in cat_cols:
        if c in X_tr_clean.columns and not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
            X_tr_clean[c] = X_tr_clean[c].astype("category")
        if c in X_va_clean.columns and not pd.api.types.is_categorical_dtype(X_va_clean[c]):
            X_va_clean[c] = X_va_clean[c].astype("category")
    return X_tr_clean, y_tr_clean, X_va_clean, y_va_clean

def _span(v, lo, hi, rel=0.2):
    a = max(lo, v*(1-rel)); b = min(hi, v*(1+rel))
    if a >= b:
        a = max(lo, v*0.9); b = min(hi, v*1.1)
    return float(a), float(b)

# --- build adaptive search space from Stage-1 top decile ----------------------
df1 = study.trials_dataframe()
df1 = df1.assign(AP = -df1["value"])
k = max(1, int(np.ceil(len(df1)*0.10)))
top = (df1.sort_values("value").head(k)).reset_index(drop=True)

best_stage1 = copy.deepcopy(study.best_trial.params)
best_lr  = float(best_stage1.get("learning_rate", 0.13))
best_md  = int(best_stage1.get("max_depth", 6))
best_nl  = int(best_stage1.get("num_leaves", 256))
best_mdl = int(best_stage1.get("min_data_in_leaf", 1500))
best_ff  = float(best_stage1.get("feature_fraction", 0.75))
best_bf  = float(best_stage1.get("bagging_fraction", 0.92))
best_l1  = float(best_stage1.get("lambda_l1", 1.0))
best_l2  = float(best_stage1.get("lambda_l2", 9.0))
best_msg = float(best_stage1.get("min_split_gain", 0.0))
use_spw  = bool(best_stage1.get("use_scale_pos_weight", True))

def _q(name, q, default):
    col = f"params_{name}"
    return float(top[col].quantile(q)) if col in top.columns and top[col].notna().any() else float(default)

lr_lo,  lr_hi  = _span(_q("learning_rate", 0.50, best_lr),   0.01, 0.5, 0.20)
md_lo,  md_hi  = int(max(3, best_md-1)), int(min(16, best_md+1))
nl_lo,  nl_hi  = int(max(31, best_nl*0.6)), int(min(2047, best_nl*1.4))
mdl_lo, mdl_hi = int(max(50, best_mdl*0.6)), int(min(10000, best_mdl*1.4))
ff_lo,  ff_hi  = _span(_q("feature_fraction", 0.50, best_ff), 0.5, 1.0, 0.15)
bf_lo,  bf_hi  = _span(_q("bagging_fraction", 0.50, best_bf), 0.5, 1.0, 0.10)
l1_lo,  l1_hi  = _span(_q("lambda_l1", 0.50, best_l1), 0.0, 10.0, 0.50)
l2_lo,  l2_hi  = _span(_q("lambda_l2", 0.50, best_l2), 0.0, 20.0, 0.30)

if "params_min_split_gain" in top.columns and top["params_min_split_gain"].notna().any():
    msg_lo, msg_hi = _span(_q("min_split_gain", 0.50, best_msg), 0.0, 0.2, 0.50)
else:
    msg_lo, msg_hi = 0.0, 0.05

with open(ART_DIR / "stage2_search_space.json","w") as f:
    json.dump({
        "learning_rate":[lr_lo, lr_hi],
        "max_depth":[md_lo, md_hi],
        "num_leaves":[nl_lo, nl_hi],
        "min_data_in_leaf":[mdl_lo, mdl_hi],
        "feature_fraction":[ff_lo, ff_hi],
        "bagging_fraction":[bf_lo, bf_hi],
        "lambda_l1":[l1_lo, l1_hi],
        "lambda_l2":[l2_lo, l2_hi],
        "min_split_gain":[msg_lo, msg_hi],
        "use_scale_pos_weight": use_spw
    }, f, indent=2)

# --- clean frames -------------------------------------------------------------
EXCLUDE = {TARGET_COL, SEQ_COL}
feat_cols = [c for c in train_df.columns if c not in EXCLUDE]
X_tr_clean, y_tr_clean, X_va_clean, y_va_clean = make_clean_frames(train_df, valid_df, feat_cols, CAT_COLS_FIXED)

TUNE2_N = 1_500_000
tr_tune2 = stratified_sample(train_df, TUNE2_N, TARGET_COL, SEED)
Xt2_clean, yt2_clean, _, _ = make_clean_frames(tr_tune2, valid_df, feat_cols, CAT_COLS_FIXED)

# imbalance policy consistent with Stage-1 best
if use_spw:
    pos = int(y_tr_clean.sum()); neg = len(y_tr_clean) - pos
    base_spw = float(neg / max(pos, 1))
else:
    base_spw = None  # will use is_unbalance

# --- objective: AP maximize (minimize -AP) + record LogLoss/WLL ---------------
def objective_stage2(trial: optuna.trial.Trial) -> float:
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "learning_rate": trial.suggest_float("learning_rate", lr_lo, lr_hi),
        "max_depth": trial.suggest_int("max_depth", md_lo, md_hi),
        "num_leaves": trial.suggest_int("num_leaves", nl_lo, nl_hi, log=True),  # log-scale
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", mdl_lo, mdl_hi, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", ff_lo, ff_hi),
        "bagging_fraction": trial.suggest_float("bagging_fraction", bf_lo, bf_hi),
        "bagging_freq": 1,
        "lambda_l1": trial.suggest_float("lambda_l1", l1_lo, l1_hi),
        "lambda_l2": trial.suggest_float("lambda_l2", l2_lo, l2_hi),
        "min_split_gain": trial.suggest_float("min_split_gain", msg_lo, msg_hi),
        "random_state": SEED, "n_jobs": -1, "verbosity": -1,
    }
    if base_spw is not None:
        params["scale_pos_weight"] = base_spw
    else:
        params["is_unbalance"] = True

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    ap_scores, nll_scores, wll_scores = [], [], []

    for tr_idx, va_idx in skf.split(Xt2_clean, yt2_clean):
        Xtr, Xcv = Xt2_clean.iloc[tr_idx], Xt2_clean.iloc[va_idx]
        ytr, ycv = yt2_clean[tr_idx], yt2_clean[va_idx]

        clf = lgb.LGBMClassifier(n_estimators=4000, **params)
        clf.fit(
            Xtr, ytr,
            eval_set=[(Xcv, ycv)],
            eval_metric=["binary_logloss", "aucpr"],
            callbacks=[lgb.early_stopping(150, verbose=False)]
        )
        p = clf.predict_proba(Xcv)[:, 1]
        ap_scores.append(average_precision_score(ycv, p))
        nll_scores.append(log_loss(ycv, p))
        wll_scores.append(weighted_log_loss(ycv, p, 0.5, 0.5))

    trial.set_user_attr("cv_logloss", float(np.mean(nll_scores)))
    trial.set_user_attr("cv_wll", float(np.mean(wll_scores)))
    return -float(np.mean(ap_scores))

study2 = optuna.create_study(direction="minimize", study_name="lgbm_stage2_ap")
study2.optimize(objective_stage2, n_trials=50, show_progress_bar=True)

df2 = study2.trials_dataframe()
topk2 = (df2.assign(AP=-df2["value"],
                    CV_LogLoss=df2["user_attrs_cv_logloss"],
                    CV_WLL=df2["user_attrs_cv_wll"])
              .sort_values("value").head(10))
display(topk2[["number","AP","CV_LogLoss","CV_WLL",
               "params_learning_rate","params_max_depth","params_num_leaves",
               "params_min_data_in_leaf","params_feature_fraction","params_bagging_fraction",
               "params_lambda_l1","params_lambda_l2","params_min_split_gain"]])


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/2581621568.py:32: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in X_tr_clean.columns and not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/2581621568.py:34: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in X_va_clean.columns and not pd.api.types.is_categorical_dtype(X_va_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/2581621568.py:32: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in X_tr_clean.columns and not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/

[I 2025-09-15 13:17:36,393] Trial 0 finished with value: -0.051093644089789246 and parameters: {'learning_rate': 0.10519257551377724, 'max_depth': 6, 'num_leaves': 93, 'min_data_in_leaf': 226, 'feature_fraction': 0.7345623372477302, 'bagging_fraction': 0.897435891635037, 'lambda_l1': 0.48409465644934, 'lambda_l2': 0.19454957103077658, 'min_split_gain': 0.028382364435816272}. Best is trial 0 with value: -0.051093644089789246.


Best trial: 1. Best value: -0.0511808:   4%|▍         | 2/50 [00:48<19:23, 24.24s/it]

[I 2025-09-15 13:18:00,484] Trial 1 finished with value: -0.051180790771154755 and parameters: {'learning_rate': 0.11823574909163727, 'max_depth': 6, 'num_leaves': 110, 'min_data_in_leaf': 216, 'feature_fraction': 0.8948901666653922, 'bagging_fraction': 0.9058737794000193, 'lambda_l1': 1.327548989900747, 'lambda_l2': 0.201703285888474, 'min_split_gain': 0.016403230252382484}. Best is trial 1 with value: -0.051180790771154755.


Best trial: 1. Best value: -0.0511808:   6%|▌         | 3/50 [01:07<17:13, 22.00s/it]

...

Best trial: 46. Best value: -0.0543901:  98%|█████████▊| 49/50 [21:20<00:26, 26.79s/it]

[I 2025-09-15 13:38:32,679] Trial 48 finished with value: -0.05365353337825992 and parameters: {'learning_rate': 0.11070769447988787, 'max_depth': 7, 'num_leaves': 131, 'min_data_in_leaf': 291, 'feature_fraction': 0.9352985518525133, 'bagging_fraction': 0.9992890910043127, 'lambda_l1': 0.7499601555791542, 'lambda_l2': 0.14917225963331196, 'min_split_gain': 0.02618775994611701}. Best is trial 46 with value: -0.054390127731745286.


Best trial: 46. Best value: -0.0543901: 100%|██████████| 50/50 [21:47<00:00, 26.15s/it]

[I 2025-09-15 13:38:59,392] Trial 49 finished with value: -0.05372257205773834 and parameters: {'learning_rate': 0.11669542168742904, 'max_depth': 7, 'num_leaves': 156, 'min_data_in_leaf': 280, 'feature_fraction': 0.9443184578923082, 'bagging_fraction': 0.9845583177548589, 'lambda_l1': 0.7137198580048357, 'lambda_l2': 0.16376881464837112, 'min_split_gain': 0.029452635982864427}. Best is trial 46 with value: -0.054390127731745286.


,number,AP,CV_LogLoss,CV_WLL,params_learning_rate,params_max_depth,params_num_leaves,params_min_data_in_leaf,params_feature_fraction,params_bagging_fraction,params_lambda_l1,params_lambda_l2,params_min_split_gain
46,46,0.054390,0.324859,0.735705,0.115461,7,131,286,0.943087,0.987869,0.834068,0.149161,0.024707
45,45,0.054369,0.294559,0.759332,0.110489,7,129,273,0.945674,0.987420,0.831823,0.163995,0.029726
34,34,0.054349,0.355920,0.717452,0.120156,7,122,285,0.954400,0.991591,0.870259,0.162126,0.030155
42,42,0.054139,0.310530,0.746925,0.113074,7,121,248,0.945357,0.973540,1.017980,0.169048,0.040637
21,21,0.053994,0.282848,0.771835,0.108323,7,124,241,0.953899,0.974846,0.949731,0.178170,0.049282
47,47,0.053989,0.324882,0.736220,0.115453,7,147,318,0.920710,0.990161,0.826833,0.138246,0.020496
14,14,0.053988,0.270702,0.784720,0.106133,7,121,245,0.949677,0.978327,1.050593,0.177366,0.049374
16,16,0.053975,0.220568,0.853742,0.095711,7,113,246,0.954576,0.965500,1.013466,0.179208,0.039417
31,31,0.053903,0.282502,0.772725,0.108304,7,123,239,0.954854,0.967101,1.095675,0.160367,0.045464
40,40,0.053878,0.219171,0.855304,0.095442,7,135,301,0.764624,0.994661,0.940250,0.179063,0.037387


In [27]:
# --- Backfill Stage-1 calibrated metrics JSON (run once) ---
import json, numpy as np
from pathlib import Path
from sklearn.metrics import average_precision_score, log_loss

ART_ROOT = Path("./artifacts/optuna_lgbm")
METRICS_JSON = ART_ROOT / "valid_gate_metrics_calibrated.json"

def _metrics(y, p):
    return {
        "AP": float(average_precision_score(y, p)),
        "LogLoss": float(log_loss(y, p)),
        "WLL": float(weighted_log_loss(y, p, 0.5, 0.5)),
        "MeanPred": float(np.mean(p)),
    }

FORCE_BACKFILL = True  

if METRICS_JSON.exists() and not FORCE_BACKFILL:
    print("[ok] JSON already exists:", METRICS_JSON)

else:
    y_va_clean = valid_df[TARGET_COL].astype(int).to_numpy()

    p_base_f = ART_ROOT / "p_valid_baseline.npy"
    p_uncal_f = ART_ROOT / "p_valid_uncal.npy"
    p_cal_f   = ART_ROOT / "p_valid_calibrated.npy"

    if not (p_base_f.exists() and p_uncal_f.exists() and p_cal_f.exists()):
        raise FileNotFoundError(
            "Saved prediction files not found. Re-run the Stage-1 Validation Gate cell "
            "that saves p_valid_baseline.npy / p_valid_uncal.npy / p_valid_calibrated.npy."
        )

    p_base  = np.load(p_base_f)
    p_uncal = np.load(p_uncal_f)
    p_cal   = np.load(p_cal_f)

    out = {
        "base": _metrics(y_va_clean, p_base),
        "uncal": _metrics(y_va_clean, p_uncal),
        "cal": _metrics(y_va_clean, p_cal),
        "base_rate": float(y_va_clean.mean()),
    }
    METRICS_JSON.write_text(json.dumps(out, indent=2))
    print("[ok] Wrote:", METRICS_JSON)


[ok] Wrote: artifacts/optuna_lgbm/valid_gate_metrics_calibrated.json


In [29]:
# =========================================================
# Stage-2 — Validation Gate with OOF Isotonic calibration
# =========================================================
import json, copy
import numpy as np, pandas as pd
from pathlib import Path
from joblib import dump
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, log_loss
from sklearn.isotonic import IsotonicRegression
import lightgbm as lgb

ART_ROOT = Path("./artifacts/optuna_lgbm")
ART_DIR  = ART_ROOT / "stage2"; ART_DIR.mkdir(parents=True, exist_ok=True)
METRICS_JSON = ART_ROOT / "valid_gate_metrics_calibrated.json"

def compute_metrics(y_true, p):
    return {
        "AP": average_precision_score(y_true, p),
        "LogLoss": log_loss(y_true, p),
        "WLL": weighted_log_loss(y_true, p, 0.5, 0.5)
    }

# prior calibrated metrics from Stage-1
try:
    prev = json.loads(METRICS_JSON.read_text())
    m_prev_cal = prev["cal"]
except FileNotFoundError:
    raise FileNotFoundError(
        f"{METRICS_JSON} not found.\n"
        "Run the backfill cell (or re-run the Stage-1 Validation Gate cell that saves "
        "'p_valid_baseline.npy', 'p_valid_uncal.npy', 'p_valid_calibrated.npy') "
        "to generate this JSON."
    )

# frames (clean)
EXCLUDE = {TARGET_COL, SEQ_COL}
feat_cols = [c for c in train_df.columns if c not in EXCLUDE]
X_tr_clean, y_tr_clean, X_va_clean, y_va_clean = make_clean_frames(train_df, valid_df, feat_cols, CAT_COLS_FIXED)

# reconstruct best params from Stage-2
best2 = copy.deepcopy(study2.best_trial.params)
if bool(study.best_trial.params.get("use_scale_pos_weight", True)):
    pos = int(y_tr_clean.sum()); neg = len(y_tr_clean) - pos
    imb = {"scale_pos_weight": float(neg / max(pos, 1))}
else:
    imb = {"is_unbalance": True}

best_full = {
    "objective":"binary", "metric":"binary_logloss", "random_state":SEED, "n_jobs":-1, "bagging_freq":1,
    **best2, **imb
}
(ART_DIR / "best_params_stage2.json").write_text(json.dumps(best_full, indent=2))

# OOF isotonic
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
oof = np.zeros_like(y_tr_clean, dtype=float)
for tr_idx, va_idx in skf.split(X_tr_clean, y_tr_clean):
    clf_cv = lgb.LGBMClassifier(n_estimators=4000, **best_full)
    clf_cv.fit(
        X_tr_clean.iloc[tr_idx], y_tr_clean[tr_idx],
        eval_set=[(X_tr_clean.iloc[va_idx], y_tr_clean[va_idx])],
        eval_metric=["binary_logloss","aucpr"],
        callbacks=[lgb.early_stopping(150, verbose=False)]
    )
    oof[va_idx] = clf_cv.predict_proba(X_tr_clean.iloc[va_idx])[:, 1]

iso = IsotonicRegression(y_min=0.0, y_max=1.0, out_of_bounds="clip").fit(oof, y_tr_clean)
dump(iso, ART_DIR / "isotonic_stage2.joblib")

# full-train fit → calibrated valid
lgb_s2 = lgb.LGBMClassifier(n_estimators=6000, **best_full)
lgb_s2.fit(
    X_tr_clean, y_tr_clean,
    eval_set=[(X_va_clean, y_va_clean)],
    eval_metric=["binary_logloss","aucpr"],
    callbacks=[lgb.early_stopping(200, verbose=False)]
)
p_uncal = lgb_s2.predict_proba(X_va_clean)[:, 1]
p_cal   = iso.transform(p_uncal)

m_uncal = compute_metrics(y_va_clean, p_uncal)
m_cal   = compute_metrics(y_va_clean, p_cal)

cmp = (pd.DataFrame([
    {"Model":"Stage-1 Calibrated (prev)", **m_prev_cal, "MeanPred": float(m_prev_cal.get("MeanPred", np.nan))},
    {"Model":"Stage-2 Uncalibrated",     **m_uncal,    "MeanPred": float(p_uncal.mean())},
    {"Model":"Stage-2 + Isotonic(OOF)",  **m_cal,      "MeanPred": float(p_cal.mean())},
])[["Model","AP","LogLoss","WLL","MeanPred"]]
 .sort_values(["LogLoss","AP"], ascending=[True, False])
 .reset_index(drop=True))
display(cmp)

delta_ap = m_cal["AP"] - m_prev_cal["AP"]
delta_ll = m_cal["LogLoss"] - m_prev_cal["LogLoss"]  # negative is better
print(f"[Stage-2 Gate] ΔAP = {delta_ap:+.6f}, ΔLogLoss = {delta_ll:+.6f} (negative is better)")

cmp.to_csv(ART_DIR / "valid_gate_comparison_stage2.csv", index=False)
with open(ART_DIR / "valid_gate_metrics_stage2.json","w") as f:
    json.dump({"stage1_calibrated": m_prev_cal, "stage2_uncal": m_uncal, "stage2_cal": m_cal,
               "delta_ap": float(delta_ap), "delta_logloss": float(delta_ll)}, f, indent=2)

from joblib import dump
dump(lgb_s2, ART_DIR / "model_stage2_best_valid_uncal.joblib")
np.save(ART_DIR / "p_valid_stage2_uncal.npy", p_uncal)
np.save(ART_DIR / "p_valid_stage2_calibrated.npy", p_cal)

AP_GAIN_MIN = 0.001
ALLOW_LL_WORSE = False
promote = (delta_ap >= AP_GAIN_MIN) and (ALLOW_LL_WORSE or (delta_ll <= 0))
print(f"[Decision] Finalize Stage-2? {'YES' if promote else 'NO'}")


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/2581621568.py:32: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in X_tr_clean.columns and not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/2581621568.py:34: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in X_va_clean.columns and not pd.api.types.is_categorical_dtype(X_va_clean[c]):


,Model,AP,LogLoss,WLL,MeanPred
0,Stage-2 + Isotonic(OOF),0.057314,0.088283,1.832454,0.019106
1,Stage-1 Calibrated (prev),0.055251,0.088486,1.835594,0.019183
2,Stage-2 Uncalibrated,0.057951,0.322766,0.722807,0.240202


[Stage-2 Gate] ΔAP = +0.002063, ΔLogLoss = -0.000203 (negative is better)
[Decision] Finalize Stage-2? YES


### Stage-2 — Summary
- Objective: refine top-decile trials to improve **AP** without hurting **LogLoss**.
- Valid (calibrated): **AP 0.0573** (+0.0021 vs S1), **LogLoss 0.0883** (−0.0002), MeanPred **0.0191** (~base 0.019).
- Decision: ✅ Proceed to Stage-3 (full-train + isotonic + holdout).


In [30]:
# =========================================================
# Stage-3 — Final model (full train) & one-shot holdout eval (no leakage)
# =========================================================
import json, copy, numpy as np, pandas as pd
from pathlib import Path
from joblib import load, dump
from sklearn.metrics import average_precision_score, log_loss
import lightgbm as lgb

# --- helpers reused ---
def prepare_lgb_frames(X_tr_src, X_te_src, cat_cols):
    X_tr_clean = X_tr_src.copy()
    X_te_clean = X_te_src.copy()
    # drop stray objects
    obj_tr = X_tr_clean.select_dtypes(include=['object']).columns.tolist()
    obj_te = X_te_clean.select_dtypes(include=['object']).columns.tolist()
    if obj_tr: X_tr_clean.drop(columns=obj_tr, inplace=True, errors="ignore")
    if obj_te: X_te_clean.drop(columns=obj_te, inplace=True, errors="ignore")
    # categorical casting
    for c in [c for c in cat_cols if c in X_tr_clean.columns]:
        if not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
            X_tr_clean[c] = X_tr_clean[c].astype("category")
    for c in [c for c in cat_cols if c in X_te_clean.columns]:
        if not pd.api.types.is_categorical_dtype(X_te_clean[c]):
            X_te_clean[c] = X_te_clean[c].astype("category")
    return X_tr_clean, X_te_clean

def compute_metrics_simple(y_true, p):
    return {
        "AP": float(average_precision_score(y_true, p)),
        "LogLoss": float(log_loss(y_true, p)),
        "WLL": float(weighted_log_loss(y_true, p, 0.5, 0.5)),
        "MeanPred": float(np.mean(p))
    }

# --- frames: full train -> holdout (no valid usage here) ---------------------
EXCLUDE = {TARGET_COL, SEQ_COL}
feat_cols = [c for c in train_df.columns if c not in EXCLUDE]

X_full_tr = train_df[feat_cols]
y_full_tr = train_df[TARGET_COL].astype(int).to_numpy()

# holdout must exist (labels optional for real competitions; here we have y_ho)
assert X_ho is not None, "holdout_df (X_ho) is required for final evaluation."
X_tr_clean, X_ho_clean = prepare_lgb_frames(X_full_tr, X_ho, CAT_COLS_FIXED)

# --- best params from Stage-2 (fallback to Stage-1 if Stage-2 absent) --------
ART_ROOT = Path("./artifacts/optuna_lgbm")
S2_DIR   = ART_ROOT / "stage2"
S2_PARAMS = S2_DIR / "best_params_stage2.json"

if S2_PARAMS.exists():
    best_params = json.loads(S2_PARAMS.read_text())
else:
    # fallback: use Stage-1 gate (saved before)
    best_params = json.loads((ART_ROOT / "best_params_valid_gate.json").read_text())

# IMPORTANT: final training must NOT use valid for early stopping
# Use a generous n_estimators; no eval_set
n_estimators_final = 6000
lgb_final = lgb.LGBMClassifier(n_estimators=n_estimators_final, **best_params)

lgb_final.fit(X_tr_clean, y_full_tr)

# --- load isotonic fitted on train-OOF (Stage-2 preferred) -------------------
ISO_S2 = S2_DIR / "isotonic_stage2.joblib"
ISO_S1 = ART_ROOT / "isotonic_stage1.joblib"
if ISO_S2.exists():
    iso = load(ISO_S2)
elif ISO_S1.exists():
    iso = load(ISO_S1)
else:
    raise FileNotFoundError("Isotonic from Stage-1/2 not found. Run Stage-2 (or Stage-1) gate first.")

# --- predict holdout once (uncalibrated -> calibrated) -----------------------
p_ho_uncal = lgb_final.predict_proba(X_ho_clean)[:, 1]
p_ho_cal   = iso.transform(p_ho_uncal)

# if holdout labels are present, compute metrics; else just save preds
results = {}
if y_ho is not None:
    m_uncal = compute_metrics_simple(y_ho, p_ho_uncal)
    m_cal   = compute_metrics_simple(y_ho, p_ho_cal)
    holdout_report = (
        pd.DataFrame([
            {"Model":"Final(Uncalibrated)", **m_uncal},
            {"Model":"Final+Isotonic(OOF from train)", **m_cal},
        ])
        .sort_values(["LogLoss","AP"], ascending=[True, False])
        .reset_index(drop=True)
    )
    display(holdout_report)

    results = {"uncal": m_uncal, "cal": m_cal}
    print(f"[Holdout base rate] {y_ho.mean():.6f}")
else:
    print("[Info] Holdout labels are missing; saving predictions only.")

# --- save artifacts ----------------------------------------------------------
OUT_DIR = ART_ROOT / "final"; OUT_DIR.mkdir(parents=True, exist_ok=True)
dump(lgb_final, OUT_DIR / "model_lgb_final.joblib")
np.save(OUT_DIR / "p_holdout_uncal.npy", p_ho_uncal)
np.save(OUT_DIR / "p_holdout_calibrated.npy", p_ho_cal)
(Path(OUT_DIR / "final_params.json")).write_text(json.dumps({"n_estimators": n_estimators_final, **best_params}, indent=2))
if results:
    (OUT_DIR / "holdout_metrics.json").write_text(json.dumps(results, indent=2))


/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/1931688018.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(X_tr_clean[c]):
/var/folders/hg/v9v4slf113jb0w4kpzx5y_zw0000gn/T/ipykernel_77034/1931688018.py:24: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(X_te_clean[c]):


,Model,AP,LogLoss,WLL,MeanPred
0,Final+Isotonic(OOF from train),0.058635,0.091558,2.060359,0.013497
1,Final(Uncalibrated),0.063388,0.216802,1.014174,0.148005


[Holdout base rate] 0.019075


### Stage-3 — Final & Holdout
- Train: full 7.28M; Calibration: isotonic on train OOF.
- Holdout (calibrated): **AP 0.0586**, **LogLoss 0.0916**, MeanPred **0.0135** (base **0.0191**) → slight under-confidence.
- Decision: use **calibrated** probs for thresholds/KPI
